In [1]:
from heapq import heappush, heappop
from collections import defaultdict

## Experience

This one was easy but took time.

Each position was given an integer.

```
#{16}{17}...{18}...{19}....{20}....{21}{22}#
#########{0}####{4}####{8} ####{12}#########
        #{1}####{5}####{9} ####{13}#
        #{2}####{6}####{10}####{14}#
        #{3}####{7}####{11}####{15}#
```

So state can be described as a list of 23 elements. 4\*4 Amphipods, and 7 unoccpied. Unoccupied is denoted as 100.


### First Approach

I (implicitly) created a dense graph. This meant each node(state) all possible neighbours were allowed.

That means that if some room had 3 Amphipods, `XYZ.` (dot being at bottom), I had allowed `XY.Z`. This created a dense graph. Similarly `XY.Z` had `X.YZ` and `XYZ.` as neightbours.

Initially, I also allowed neighbours that violated rules of the Amphipod. E.g. if A is at hallway X, it was allowed to roam at X+2 or X-2 (provided that is was not occupied). This violated the rule that Amphipod's are allowed to stop only once in the hallway.

This approch quickly filled memory (7GB in around a minute). So had to be improved. I am really embarassed that I didn't read the rules carefully, they have given so many hints.

### Second approach (mapping of English rules to state space)

* You can only move from room to hallway, and hallway to room.
* You can only move to bottomest allowed position if you are moving into a room.
* You can't enter the room if it is occupied by some other variety of amphipod.

At first, I felt that the rules described in the puzzle were arbitrary. But after my failed first approach, I realized that these rules made the statespace quite small so that it can be explored with small enough computer.

### Possible improvements:
A* search. A variation of L1 distance as heuristic.

In [2]:
corridor2base = {
    (16, 0): 3, (16, 1): 5, (16, 2): 7, (16, 3): 9, 
    (17, 0): 2, (17, 1): 4, (17, 2): 6, (17, 3): 8,
    (18, 0): 2, (18, 1): 2, (18, 2): 4, (18, 3): 6, 
    (19, 0): 4, (19, 1): 2, (19, 2): 2, (19, 3): 4,
    (20, 0): 6, (20, 1): 4, (20, 2): 2, (20, 3): 2,
    (21, 0): 8, (21, 1): 6, (21, 2): 4, (21, 3): 2, 
    (22, 0): 9, (22, 1): 7, (22, 2): 5, (22, 3): 3,
}

def is_end(s):
    for i in range(16):
        if s[i] == 100 or s[i] != i//4: return False
    return True

def can_exit(s, hall):
    base = hall//4 * 4
    for i in range(hall-1, base-1, -1):
        if s[i] != 100: return False
    return True

def find_deepest(s, base, a):
    for i in range(base+3, base-1, -1):
        if s[i] == 100: return i
        if s[i] != a: return None
    return None

costs = [1, 10, 100, 1000]

def neighbours(s):
    s = list(s)
    for i, a in enumerate(s):
        if a == 100: continue # Not occupied
        if i < 16: # Room
            if not can_exit(s, i): continue
            base = i//4
            left_cor = 17 + base
            d = i-base*4
            for k in range(left_cor, 15, -1):
                d += 2
                if s[k] != 100: break
                if k == 16: d -= 1
                s[k], s[i] = a, 100
                yield tuple(s), d*costs[a]
                s[k], s[i] = 100, a                
            d = i-base*4
            for k in range(left_cor+1, 23):
                d += 2
                if s[k] != 100: break
                s[k], s[i] = a, 100
                if k == 22: d -= 1
                yield tuple(s), d*costs[a]
                s[k], s[i] = 100, a
        else: # Corridor
            base = a*4
            k = find_deepest(s, base, a)
            if k is None: continue
            r = 17+a
            if r >= i:
                r = r+1
                l = i+1
                dir = 1
            else:
                l = i-1
                dir = -1
                  
            can_enter = True
            for c in range(l, r, dir):
                if s[c] != 100:
                    can_enter = False
                    break
            if not can_enter: continue
                
            s[k], s[i] = a, 100
            yield tuple(s), (corridor2base[i, a]+(k-base))*costs[a]
            s[k], s[i] = 100, a
            
                

In [3]:
inputs = {
    'test_part1': 'BAAACDBBBCCCDADD',
    'test_part2': 'BDDACCBDBBACDACA',
    'Part1': 'DDAABABBCBCCCADD',
    'Part2': 'DDDDBCBACBABCACA',
    'debug1': 'AAAABBBBCCCCDDDD'
}

In [4]:
def solve(part):
    initial = tuple(['ABCD'.index(c) for c in inputs[part]] + [100]*7)
    heap = [(0, initial)]
    found = {}
    cost_d = defaultdict(lambda: float('inf'))
    while heap:
        cost, node = heappop(heap)
        if is_end(node):
            print(part, cost)
            break
        if node in found: continue
        found[node] = cost

        for n, c in neighbours(node):
            if n not in found and cost+c < cost_d[n]:
                cost_d[n] = cost+c
                heappush(heap, (cost+c, n))

In [5]:
solve('Part1')

Part1 19019


In [6]:
solve('Part2')

Part2 47533


## For Debugging

In [7]:
def pprint(node):
    m = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 100: '.'}
    node = tuple(m[n] for n in node)
#     s = """
# #############
# #{16}{17}.{18}.{19}.{20}.{21}{22}#
# ###{0}#{4}#{8}#{12}###
#   #{1}#{5}#{9}#{13}#
#   #########
# """
    s = """
#############
#{16}{17}.{18}.{19}.{20}.{21}{22}#
###{0}#{4}#{8}#{12}###
  #{1}#{5}#{9}#{13}#
  #{2}#{6}#{10}#{14}#
  #{3}#{7}#{11}#{15}#
  #########
"""
 
    print(s.format(*node))